In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import json
import re
from requests import get
import collections
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
rank_df = pd.read_csv("fifa_ranking.csv")

In [4]:
rank_groups = rank_df.groupby('rank_date')

In [5]:
wc2006_rankings = rank_groups.get_group('5/17/06')
wc2010_rankings = rank_groups.get_group('5/26/10')
wc2014_rankings = rank_groups.get_group('5/8/14')
wc2018_rankings = rank_groups.get_group('5/17/18')

In [6]:
tourney_df = pd.read_csv('WorldCupMatches.csv', encoding = 'ISO-8859-1')
tourney_df

,Year,Datetime,Stage,Unnamed: 3,Unnamed: 4,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930,13 Jul 1930 - 15:00,Group 1,NaN,NaN,Pocitos,Montevideo,France,4,1,Mexico,,4444.0,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX
1,1930,13 Jul 1930 - 15:00,Group 4,NaN,NaN,Parque Central,Montevideo,USA,3,0,Belgium,,18346.0,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL
2,1930,14 Jul 1930 - 12:45,Group 2,NaN,NaN,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,24059.0,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA
3,1930,14 Jul 1930 - 14:50,Group 3,NaN,NaN,Pocitos,Montevideo,Romania,3,1,Peru,,2549.0,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER
4,1930,15 Jul 1930 - 16:00,Group 1,NaN,NaN,Parque Central,Montevideo,Argentina,1,0,France,,23409.0,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA
5,1930,16 Jul 1930 - 14:45,Group 1,NaN,NaN,Parque Central,Montevideo,Chile,3,0,Mexico,,9249.0,1,0,CRISTOPHE Henry (BEL),APHESTEGUY Martin (URU),LANGENUS Jean (BEL),201,1095,CHI,MEX
6,1930,17 Jul 1930 - 12:45,Group 2,NaN,NaN,Parque Central,Montevideo,Yugoslavia,4,0,Bolivia,,18306.0,0,0,MATEUCCI Francisco (URU),LOMBARDI Domingo (URU),WARNKEN Alberto (CHI),201,1092,YUG,BOL
7,1930,17 Jul 1930 - 14:45,Group 4,NaN,NaN,Parque Central,Montevideo,USA,3,0,Paraguay,,18306.0,2,0,MACIAS Jose (ARG),APHESTEGUY Martin (URU),TEJADA Anibal (URU),201,1097,USA,PAR
8,1930,18 Jul 1930 - 14:30,Group 3,NaN,NaN,Estadio Centenario,Montevideo,Uruguay,1,0,Peru,,57735.0,0,0,LANGENUS Jean (BEL),BALWAY Thomas (FRA),CRISTOPHE Henry (BEL),201,1099,URU,PER
9,1930,19 Jul 1930 - 12:50,Group 1,NaN,NaN,Estadio Centenario,Montevideo,Chile,1,0,France,,2000.0,0,0,TEJADA Anibal (URU),LOMBARDI Domingo (URU),REGO Gilberto (BRA),201,1094,CHI,FRA


In [7]:
home_team_result = []
diff = tourney_df['Home Team Goals'] - tourney_df['Away Team Goals']
for i in range(len(diff)):
    if diff[i] > 0:
        home_team_result.append('W')
    elif diff[i] == 0:
        home_team_result.append('D')
    else:
        home_team_result.append('L')

In [8]:
for i in range(len(tourney_df)):
    if tourney_df['Win conditions'][i] != ' ':
        if tourney_df['Home Team Name'][i] in tourney_df['Win conditions'][i]:
            home_team_result[i] = 'W'
        else:
            home_team_result[i] = 'L'

tourney_df['Home Team Result'] = home_team_result

In [9]:
tourney_groups = tourney_df.groupby('Year')

In [10]:
wc_2006_tourney = tourney_groups.get_group(2006)
wc_2010_tourney = tourney_groups.get_group(2010)
wc_2014_tourney = tourney_groups.get_group(2014)

In [11]:
def rank_game (tourney, rank):
    rank['Home Team Initials'] = rank['country_abrv']
    rank['Away Team Initials'] = rank['country_abrv']
    rank_abv = rank[['rank', 'Home Team Initials', 'Away Team Initials']]
    merge_home = tourney.merge(rank_abv[['rank', 'Home Team Initials']], 
                               how = 'left', on = 'Home Team Initials')
    merge_home = merge_home.rename(columns={'rank': 'Home Rank'})
    merge_both = merge_home.merge(rank_abv[['rank', 'Away Team Initials']], 
                                  how = 'left', on = 'Away Team Initials') 
    merge_both = merge_both.rename(columns={'rank': 'Away Rank'})
    return merge_both

In [12]:
merged_2006 = rank_game(wc_2006_tourney, wc2006_rankings)
merged_2010 = rank_game(wc_2010_tourney, wc2010_rankings)
merged_2014 = rank_game(wc_2014_tourney, wc2014_rankings)

In [13]:
step1 = merged_2006.append(merged_2010, ignore_index = True)

In [14]:
merged_set = step1.append(merged_2014, ignore_index = True)

In [15]:
merged_set

,Year,Datetime,Stage,Unnamed: 3,Unnamed: 4,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Home Team Result,Home Rank,Away Rank
0,2006,09 Jun 2006 - 18:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Munich",Munich,Germany,4,2,Costa Rica,,66000.0,2,1,ELIZONDO Horacio (ARG),GARCIA Dario (ARG),OTERO Rodolfo (ARG),97410100,97410001,GER,CRC,W,19,26
1,2006,09 Jun 2006 - 21:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,Poland,0,2,Ecuador,,52000.0,0,1,KAMIKAWA Toru (JPN),HIROSHIMA Yoshikazu (JPN),KIM Dae Young (KOR),97410100,97410002,POL,ECU,L,29,39
2,2006,10 Jun 2006 - 15:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Frankfurt",Frankfurt/Main,England,1,0,Paraguay,,48000.0,1,0,RODRIGUEZ Marco (MEX),CAMARGO Jose Luis (MEX),LEAL Leonel (CRC),97410100,97410003,ENG,PAR,W,10,33
3,2006,10 Jun 2006 - 18:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Dortmund",Dortmund,"rn"">Trinidad and Tobago",0,0,Sweden,,62959.0,0,0,MAIDIN Shamsul (SIN),PERMPANICH Prachya (THA),GHULOUM Eisa (UAE),97410100,97410004,TRI,SWE,D,47,16
4,2006,10 Jun 2006 - 21:00,Group C,NaN,NaN,"FIFA World Cup Stadium, Hamburg",Hamburg,Argentina,2,1,C¥Ë_te d'Ivoire,,49480.0,2,0,DE BLEECKERE Frank (BEL),HERMANS Peter (BEL),VROMANS Walter (BEL),97410100,97410005,ARG,CIV,W,9,32
5,2006,11 Jun 2006 - 15:00,Group C,NaN,NaN,Zentralstadion,Leipzig,"rn"">Serbia and Montenegro",0,1,Netherlands,,43000.0,0,1,MERK Markus (GER),SCHRAER Christian (GER),SALVER Jan-Hendrik (GER),97410100,97410006,SCG,NED,L,44,3
6,2006,11 Jun 2006 - 18:00,Group D,NaN,NaN,Franken-Stadion,Nuremberg,Mexico,3,1,Iran,,41000.0,1,1,ROSETTI Roberto (ITA),COPELLI Cristiano (ITA),STAGNOLI Alessandro (ITA),97410100,97410007,MEX,IRN,W,4,23
7,2006,11 Jun 2006 - 21:00,Group D,NaN,NaN,"FIFA World Cup Stadium, Cologne",Cologne,Angola,0,1,Portugal,,45000.0,0,1,LARRIONDA Jorge (URU),RIAL Walter (URU),FANDINO Pablo (URU),97410100,97410008,ANG,POR,L,57,7
8,2006,12 Jun 2006 - 15:00,Group F,NaN,NaN,Fritz-Walter-Stadion,Kaiserslautern,Australia,3,1,Japan,,46000.0,0,1,ABD EL FATAH Essam (EGY),DANTE Dramane (MLI),NDOYE Mamadou (SEN),97410100,97410012,AUS,JPN,W,42,18
9,2006,12 Jun 2006 - 18:00,Group E,NaN,NaN,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,USA,0,3,Czech Republic,,52000.0,0,2,AMARILLA Carlos (PAR),ANDINO Amelio (PAR),BERNAL Manuel (PAR),97410100,97410010,USA,CZE,L,5,2


In [16]:
for i in range(len(merged_set['Home Team Name'])):
    if '>' in merged_set['Home Team Name'][i] :
        merged_set['Home Team Name'][i] = merged_set['Home Team Name'][i][4:]
    if 'Ivoire' in merged_set['Home Team Name'][i]:
        merged_set['Home Team Name'][i] = 'Ivory Coast'

for i in range(len(merged_set['Away Team Name'])):
    if '>' in merged_set['Away Team Name'][i] :
        merged_set['Away Team Name'][i] = merged_set['Away Team Name'][i][4:]
    if 'Ivoire' in merged_set['Away Team Name'][i]:
        merged_set['Away Team Name'][i] = 'Ivory Coast'

In [17]:
country_factors = pd.read_csv("gdp_rain_temp.csv", encoding = 'ISO-8859-1')
country_factors

,Unnamed: 0,country,temp_in_celcius,rain_mm_per_year,gdp(millions_dollars),population,gdp(dollars),gdp_dollar_per_person
0,0,Burkina Faso,28.25,748,13187,19751651,1.318700e+10,667.640391
1,1,Mali,28.25,282,14998,19107706,1.499800e+10,784.918922
2,2,Djibouti,28,220,2082,971408,2.082000e+09,2143.280681
3,3,Senegal,27.85,686,16057,16294270,1.605700e+10,985.438439
4,4,Mauritania,27.65,92,4985,4540068,4.985000e+09,1098.001175
5,5,Maldives,27.65,1972,4520,444259,4.520000e+09,10174.245200
6,6,Benin,27.55,1039,9410,11485674,9.410000e+09,819.281481
7,7,The Gambia,27.5,836,1038,2163765,1.038000e+09,479.719378
8,8,Ghana,27.2,1187,47032,29463643,4.703200e+10,1596.272396
9,9,Bahrain,27.15,83,34895,1566993,3.489500e+10,22268.765720


In [18]:
def country_game (country_facts, tourney):
    country_facts['Home Team Name'] = country_facts['country']
    merge_home = tourney.merge(country_facts[['Home Team Name', 'temp_in_celcius', 'rain_mm_per_year', 'gdp(millions_dollars)',
                                              'population', 'gdp_dollar_per_person']], 
                               how = 'left', on = 'Home Team Name')
    merge_home = merge_home.rename(columns={'temp_in_celcius': 'Home Temp', 
                                            'rain_mm_per_year': 'Home Rain',
                                            'gdp(millions_dollars)': 'Home GDP (millions)',
                                            'population': 'Home Population',
                                            'gdp_dollar_per_person': 'Home GDP per Person (dollars)'})
    country_facts['Away Team Name'] = country_facts['country']
    merge_both = merge_home.merge(country_facts[['Away Team Name', 'temp_in_celcius', 'rain_mm_per_year', 'gdp(millions_dollars)',
                                                 'population', 'gdp_dollar_per_person']], 
                                  how = 'left', on = 'Away Team Name')
    merge_both = merge_both.rename(columns={'temp_in_celcius': 'Away Temp', 
                                            'rain_mm_per_year': 'Away Rain',
                                            'gdp(millions_dollars)': 'Away GDP (millions)',
                                            'population': 'Away Population',
                                            'gdp_dollar_per_person': 'Away GDP per Person (dollars)'})
    return merge_both

In [21]:
merged_country_info = country_game(country_factors, merged_set)
merged_country_info

,Year,Datetime,Stage,Unnamed: 3,Unnamed: 4,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Home Team Result,Home Rank,Away Rank,Home Temp,Home Rain,Home GDP (millions),Home Population,Home GDP per Person (dollars),Away Temp,Away Rain,Away GDP (millions),Away Population,Away GDP per Person (dollars)
0,2006,09 Jun 2006 - 18:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Munich",Munich,Germany,4,2,Costa Rica,,66000.0,2,1,ELIZONDO Horacio (ARG),GARCIA Dario (ARG),OTERO Rodolfo (ARG),97410100,97410001,GER,CRC,W,19,26,8.5,700,3684816,82293457,44776.536730,24.8,2926,58056,4953199,11720.910060
1,2006,09 Jun 2006 - 21:00,Group A,NaN,NaN,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,Poland,0,2,Ecuador,,52000.0,0,1,KAMIKAWA Toru (JPN),HIROSHIMA Yoshikazu (JPN),KIM Dae Young (KOR),97410100,97410002,POL,ECU,L,29,39,7.85,600,524886,38104832,13774.788460,21.85,2087,102311,16863425,6067.035611
2,2006,10 Jun 2006 - 15:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Frankfurt",Frankfurt/Main,England,1,0,Paraguay,,48000.0,1,0,RODRIGUEZ Marco (MEX),CAMARGO Jose Luis (MEX),LEAL Leonel (CRC),97410100,97410003,ENG,PAR,W,10,33,8.45,1220,2624529,66573504,39423.026310,23.55,1130,29619,6896908,4294.533144
3,2006,10 Jun 2006 - 18:00,Group B,NaN,NaN,"FIFA World Cup Stadium, Dortmund",Dortmund,Trinidad and Tobago,0,0,Sweden,,62959.0,0,0,MAIDIN Shamsul (SIN),PERMPANICH Prachya (THA),GHULOUM Eisa (UAE),97410100,97410004,TRI,SWE,D,47,16,25.75,2200,20300,1372598,14789.472230,2.1,624,538575,9982709,53950.786300
4,2006,10 Jun 2006 - 21:00,Group C,NaN,NaN,"FIFA World Cup Stadium, Hamburg",Hamburg,Argentina,2,1,Ivory Coast,,49480.0,2,0,DE BLEECKERE Frank (BEL),HERMANS Peter (BEL),VROMANS Walter (BEL),97410100,97410005,ARG,CIV,W,9,32,14.8,591,637717,44688864,14270.154640,26.35,1348,40360,24905843,1620.503269
5,2006,11 Jun 2006 - 15:00,Group C,NaN,NaN,Zentralstadion,Leipzig,Serbia and Montenegro,0,1,Netherlands,,43000.0,0,1,MERK Markus (GER),SCHRAER Christian (GER),SALVER Jan-Hendrik (GER),97410100,97410006,SCG,NED,L,44,3,10.55,680,41471,8762027,4733.037230,9.25,778,825745,17084459,48333.107880
6,2006,11 Jun 2006 - 18:00,Group D,NaN,NaN,Franken-Stadion,Nuremberg,Mexico,3,1,Iran,,41000.0,1,1,ROSETTI Roberto (ITA),COPELLI Cristiano (ITA),STAGNOLI Alessandro (ITA),97410100,97410007,MEX,IRN,W,4,23,21,752,1149236,130759074,8788.957927,17.25,228,431920,82011735,5266.563377
7,2006,11 Jun 2006 - 21:00,Group D,NaN,NaN,"FIFA World Cup Stadium, Cologne",Cologne,Angola,0,1,Portugal,,45000.0,0,1,LARRIONDA Jorge (URU),RIAL Walter (URU),FANDINO Pablo (URU),97410100,97410008,ANG,POR,L,57,7,21.55,1010,124209,30774205,4036.140008,15.15,854,218064,10291196,21189.373910
8,2006,12 Jun 2006 - 15:00,Group F,NaN,NaN,Fritz-Walter-Stadion,Kaiserslautern,Australia,3,1,Japan,,46000.0,0,1,ABD EL FATAH Essam (EGY),DANTE Dramane (MLI),NDOYE Mamadou (SEN),97410100,97410012,AUS,JPN,W,42,18,21.65,534,1379548,24772247,55689.255800,11.15,1668,4872135,127185332,38307.365510
9,2006,12 Jun 2006 - 18:00,Group E,NaN,NaN,"FIFA World Cup Stadium, Gelsenkirchen",Gelsenkirchen,USA,0,3,Czech Republic,,52000.0,0,2,AMARILLA Carlos (PAR),ANDINO Amelio (PAR),BERNAL Manuel (PAR),97410100,97410010,USA,CZE,L,5,2,8.55,715,19390600,326766748,59340.799270,7.55,677,213189,10625250,20064.374960


In [23]:
merged_country_info.to_csv('wc2006,10,14_country_and_rank.csv')